First subsample (stratified) in case we don't have enough time to make enough requests.

In [88]:
# Stratified sampling script for rental listings data
# Performs stratified sampling by suburb, year, and quarter to reduce data to 50%

import pandas as pd
import numpy as np

# Load the data
print("Loading data...")
df = pd.read_csv('../data/curated/rent_features/cleaned_listings.csv')
print(f"Original dataset shape: {df.shape}")
print(f"Original dataset size: {len(df):,} records")

# Display data distribution before sampling
print("\nData distribution before sampling:")
print("Year distribution:")
print(df['year'].value_counts().sort_index())
print("\nQuarter distribution:")
print(df['quarter'].value_counts().sort_index())
print(f"\nUnique suburbs: {df['suburb'].nunique()}")
print(f"Unique year-quarter combinations: {df[['year', 'quarter']].drop_duplicates().shape[0]}")

# Create stratification groups
print("\nCreating stratification groups...")
df['strata'] = df['suburb'].astype(str) + '_' + df['year'].astype(str) + '_' + df['quarter'].astype(str)
strata_counts = df['strata'].value_counts()
print(f"Number of strata: {len(strata_counts)}")
print(f"Strata size distribution:")
print(strata_counts.describe())

# Perform stratified sampling
print("\nPerforming stratified sampling...")
sampled_dfs = []

for stratum in df['strata'].unique():
    stratum_data = df[df['strata'] == stratum]
    
    # Calculate sample size (50% of stratum)
    sample_size = max(1, len(stratum_data) // 2)  # Ensure at least 1 record per stratum
    
    # If stratum has only 1 record, keep it
    if len(stratum_data) == 1:
        sampled_dfs.append(stratum_data)
    else:
        # Random sampling within stratum
        sampled_stratum = stratum_data.sample(n=sample_size, random_state=42)
        sampled_dfs.append(sampled_stratum)

# Combine all sampled strata
sampled_df = pd.concat(sampled_dfs, ignore_index=True)

# Remove the temporary strata column
sampled_df = sampled_df.drop('strata', axis=1)

print(f"\nSampled dataset shape: {sampled_df.shape}")
print(f"Sampled dataset size: {len(sampled_df):,} records")
print(f"Reduction: {((len(df) - len(sampled_df)) / len(df) * 100):.1f}%")

# Display data distribution after sampling
print("\nData distribution after sampling:")
print("Year distribution:")
print(sampled_df['year'].value_counts().sort_index())
print("\nQuarter distribution:")
print(sampled_df['quarter'].value_counts().sort_index())
print(f"\nUnique suburbs: {sampled_df['suburb'].nunique()}")

# Verify stratification is maintained
print("\nVerifying stratification:")
strata_verification = sampled_df.groupby(['suburb', 'year', 'quarter']).size()
print(f"Strata with data after sampling: {len(strata_verification)}")
print("Sample of strata sizes:")
print(strata_verification.head(10))


# Create a new column for isochrones and fill with NaN, we expect 6 columns (3 for driving, 3 for walking with fixed time of 5, 10, 15 minutes respectively)
sampled_df[['driving_5min', 'driving_10min', 'driving_15min', 'walking_5min', 'walking_10min', 'walking_15min']] = np.nan

# Save the sampled data
output_path = '../data/curated/rent_features/cleaned_listings_sampled.csv'
sampled_df.to_csv(output_path, index=False)
print(f"\nSampled data saved to: {output_path}")

# Summary statistics
print("\nSummary:")
print(f"Original records: {len(df):,}")
print(f"Sampled records: {len(sampled_df):,}")
print(f"Sampling ratio: {len(sampled_df)/len(df):.2%}")
print(f"Records removed: {len(df) - len(sampled_df):,}")


Loading data...


/var/folders/xj/ny4zmnqd609bv2rpjzzr11rm0000gn/T/ipykernel_19525/4289311160.py:9: DtypeWarning: Columns (1,15,16,17,19,21,22,24,25,31,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/curated/rent_features/cleaned_listings.csv')


Original dataset shape: (25687, 40)
Original dataset size: 25,687 records

Data distribution before sampling:
Year distribution:
year
2022     2620
2023      340
2024     4065
2025    18662
Name: count, dtype: int64

Quarter distribution:
quarter
3      3357
6      5937
9     15208
12     1185
Name: count, dtype: int64

Unique suburbs: 988
Unique year-quarter combinations: 15

Creating stratification groups...
Number of strata: 2721
Strata size distribution:
count    2721.000000
mean        9.440279
std        24.438703
min         1.000000
25%         1.000000
50%         3.000000
75%        11.000000
max       846.000000
Name: count, dtype: float64

Performing stratified sampling...

Sampled dataset shape: (12882, 40)
Sampled dataset size: 12,882 records
Reduction: 49.9%

Data distribution after sampling:
Year distribution:
year
2022    1295
2023     175
2024    2090
2025    9322
Name: count, dtype: int64

Quarter distribution:
quarter
3     1694
6     2977
9     7596
12     615
Name

Because the ORS isochrone is limited to 500 requests. We will rerun this notebook and update the `curated/rent_features/cleaned_listings.csv` as we impute. 

In [89]:
# First read in the subsampled data
df = pd.read_csv('../data/curated/rent_features/cleaned_listings_sampled.csv')



df[['property_id', 'coordinates','driving_5min', 'driving_10min', 'driving_15min', 'walking_5min', 'walking_10min', 'walking_15min']].to_csv('../data/raw/missing_isochrones.csv', index=False)


# Running API request below

In [96]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import geopandas as gpd
import os 


# Import the GeoUtils class from utils/geo.py
from utils.geo import GeoUtils
from dotenv import load_dotenv

load_dotenv()
APIKEY1 = os.getenv('ORS_API_KEY1')
APIKEY2 = os.getenv('ORS_API_KEY2')
APIKEY3 = os.getenv('ORS_API_KEY3')
APIKEY4 = os.getenv('ORS_API_KEY4')
APIKEY5 = os.getenv('ORS_API_KEY5')
# Initialize GeoUtils 
geoutils = GeoUtils(ors_api_key=APIKEY1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
OpenRouteService client initialized successfully.


In [90]:
# read in the missing isochrones using geopandas
gdf = gpd.read_file('../data/raw/missing_isochrones.csv') 

# get the first 500 
gdf_tmp = gdf.head(500) 

In [91]:
# Convert coordinates to Point objects from shapely
from shapely.geometry import Point
from shapely import wkt

# Check the format of the coordinates column
sample_coords = gdf_tmp['coordinates'].iloc[0]
print(f"Sample coordinate format: {sample_coords}")

# Convert WKT POINT strings to Point objects
if isinstance(sample_coords, str) and sample_coords.startswith('POINT'):
    # Handle WKT POINT format: "POINT (lon lat)"
    gdf_tmp['geometry'] = gdf_tmp['coordinates'].apply(
        lambda coord: wkt.loads(coord)
    )
    print("Successfully converted WKT POINT strings to Point objects")
else:
    print("Unexpected coordinate format. Expected WKT POINT format.")

# Verify the conversion worked
print(f"\nCreated geometry column with {gdf_tmp['geometry'].notna().sum()} valid points")
print("Sample geometry objects:")
print(gdf_tmp['geometry'].head())
print(f"\nGeometry type: {type(gdf_tmp['geometry'].iloc[0])}")


Sample coordinate format: POINT (-37.8058235 144.9940691)
Successfully converted WKT POINT strings to Point objects

Created geometry column with 500 valid points
Sample geometry objects:
0    POINT (-37.8058235 144.9940691)
1     POINT (-37.8111571 145.008908)
2    POINT (-37.8041915 144.9956615)
3     POINT (-37.8111571 145.008908)
4    POINT (-37.8109899 145.0067059)
Name: geometry, dtype: object

Geometry type: <class 'shapely.geometry.point.Point'>


/var/folders/xj/ny4zmnqd609bv2rpjzzr11rm0000gn/T/ipykernel_19525/2102773548.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_tmp['geometry'] = gdf_tmp['coordinates'].apply(


In [ ]:
driving_result = gdf_tmp['geometry'].apply(lambda x: geoutils.get_isochrone_with_delay(profile="driving-car", range_values=[300, 600, 900], coordinate=x))
gdf_tmp.loc[:, ['driving_5min', 'driving_10min', 'driving_15min'] ] = pd.DataFrame(driving_result.tolist(), columns=['driving_5min', 'driving_10min', 'driving_15min']).values

walking_result = gdf_tmp['geometry'].apply(lambda x: geoutils.get_isochrone_with_delay(profile="foot-walking", range_values=[300, 600, 900], coordinate=x))
gdf_tmp.loc[:, ['walking_5min', 'walking_10min', 'walking_15min'] ] = pd.DataFrame(walking_result.tolist(), columns=['walking_5min', 'walking_10min', 'walking_15min']).values

Getting driving-car isochrone for coordinate: (144.9940691, -37.8058235)
{'type': 'FeatureCollection', 'bbox': [144.93386, -37.859372, 145.118402, -37.747524], 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 300.0, 'center': [144.99389447507681, -37.80580526792242]}, 'geometry': {'coordinates': [[[144.976522, -37.810288], [144.978183, -37.811026], [144.984651, -37.815904], [144.985887, -37.818562], [144.987568, -37.821397], [144.989112, -37.823206], [144.989171, -37.823213], [144.994326, -37.821505], [145.000247, -37.82041], [145.008361, -37.81811], [145.010052, -37.817995], [145.011985, -37.816807], [145.014808, -37.813881], [145.014919, -37.812973], [145.01523, -37.810306], [145.012953, -37.802426], [145.010214, -37.800089], [145.00282, -37.795083], [145.002688, -37.794636], [145.002304, -37.792999], [144.995731, -37.790734], [144.995667, -37.790726], [144.988999, -37.790029], [144.988801, -37.790008], [144.9883, -37.790192], [144.986844, -37.791105], [144.

/Users/jackshee/University/MAST30034 Applied Data Science/project2/venv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


{'type': 'FeatureCollection', 'bbox': [144.93634, -37.852572, 145.12033, -37.74631], 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 300.0, 'center': [145.00014988569606, -37.799238081770554]}, 'geometry': {'coordinates': [[[144.979364, -37.798542], [144.981814, -37.802665], [144.982686, -37.803453], [144.984544, -37.80477], [144.990102, -37.811765], [144.99285, -37.813595], [144.992968, -37.813606], [145.000914, -37.813577], [145.004458, -37.812746], [145.005909, -37.811364], [145.013609, -37.809303], [145.015377, -37.809642], [145.01958, -37.810231], [145.019776, -37.810251], [145.019923, -37.810235], [145.023208, -37.808623], [145.024433, -37.807567], [145.024524, -37.807097], [145.024527, -37.80708], [145.024608, -37.806638], [145.024113, -37.802957], [145.023513, -37.801795], [145.019974, -37.801133], [145.012954, -37.79915], [145.012304, -37.798867], [145.011891, -37.798805], [145.010726, -37.798649], [145.0052, -37.795884], [145.005968, -37.789423], [1

/Users/jackshee/University/MAST30034 Applied Data Science/project2/venv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/jackshee/University/MAST30034 Applied Data Science/project2/venv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/jackshee/University/MAST30034 Applied Data Science/project2/venv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/jackshee/University/MAST30034 Applied Data Science/project2/venv/lib/python3.10/site-packages/openrouteservice/client.py:211: 

{'type': 'FeatureCollection', 'bbox': [144.936753, -37.85128, 145.115972, -37.747562], 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 300.0, 'center': [145.00202256680123, -37.79975692487193]}, 'geometry': {'coordinates': [[[144.981586, -37.800868], [144.981967, -37.801756], [144.983561, -37.803797], [144.989614, -37.809844], [144.991536, -37.811707], [144.992982, -37.812921], [144.993227, -37.812946], [144.993985, -37.813016], [145.000319, -37.813113], [145.002075, -37.812511], [145.002764, -37.811158], [145.003463, -37.80994], [145.01049, -37.808331], [145.012763, -37.811022], [145.013162, -37.811065], [145.021072, -37.808515], [145.021951, -37.80734], [145.021981, -37.807276], [145.022054, -37.807088], [145.023787, -37.802629], [145.023481, -37.802074], [145.02279, -37.801788], [145.014251, -37.799905], [145.013852, -37.799654], [145.012954, -37.79915], [145.012304, -37.798867], [145.011891, -37.798805], [145.010726, -37.798649], [145.009846, -37.798793],

In [51]:
# convert result into three columns by splitting the list of polygons 
# drop null values
result = result.dropna()
result.apply(lambda x: (x[0], x[1], x[2]))

0      (POLYGON ((144.976522 -37.810288, 144.978183 -...
1      (POLYGON ((144.983792 -37.809152, 144.984422 -...
2      (POLYGON ((144.978882 -37.805563, 144.978845 -...
3      (POLYGON ((144.983792 -37.809152, 144.984422 -...
4      (POLYGON ((144.979992 -37.808956, 144.980937 -...
                             ...                        
494    (POLYGON ((147.214933 -36.775441, 147.215281 -...
495    (POLYGON ((146.96751 -37.848101, 146.971103 -3...
496    (POLYGON ((145.109838 -38.04644, 145.110028 -3...
497    (POLYGON ((145.105053 -38.038629, 145.105422 -...
498    (POLYGON ((145.108349 -38.044669, 145.109945 -...
Name: geometry, Length: 499, dtype: object

In [82]:
gdf_tmp.loc[1:,['driving_5min', 'driving_10min', 'driving_15min'] ] = pd.DataFrame(result.tolist(), columns=['driving_5min', 'driving_10min', 'driving_15min']).values

In [87]:
# We read in the cleaned_listings_sampled.csv file that needs to be imputed with the API result
df = pd.read_csv('../data/curated/rent_features/cleaned_listings_sampled.csv')

print("Number of missing coordinates before imputation: ", df['driving_5min'].isnull().sum())

# coord_mapping = df_tmp.set_index('property_id')['coordinates'].to_dict()

# mask = df['coordinates'].isna()

# df.loc[mask, 'coordinates'] = df.loc[mask, 'property_id'].map(coord_mapping)

# print("Number of missing coordinates after imputation: ", df['coordinates'].isnull().sum())

# # save the imputed results
# df.to_csv('../data/curated/rent_features/cleaned_listings.csv', index=False)



KeyError: 'driving_5min'

In [83]:
gdf_tmp

,property_id,bedrooms,bathrooms,car_spaces,property_type,land_area,property_features,suburb,postcode,year,...,url,coordinates,weekly_rent,driving_5min,driving_10min,driving_15min,walking_5min,walking_10min,walking_15min,geometry
0,17757280,2,,1.0,house,,2 1,abbotsford,3067,2025,...,httpswwwdomaincomau42-raphael-street-abbotsfor...,POINT (-37.8058235 144.9940691),850.0,,,,,,,POINT (-37.8058235 144.9940691)
1,17706353,1,,1.0,apartment unit flat,,1 1,abbotsford,3067,2025,...,httpswwwdomaincomau629-631-victoria-street-abb...,POINT (-37.8111571 145.008908),520.0,"POLYGON ((144.976522 -37.810288, 144.978183 -3...","POLYGON ((144.954075 -37.805642, 144.954154 -3...","POLYGON ((144.936515 -37.785747, 144.936694 -3...",,,,POINT (-37.8111571 145.008908)
2,17719363,3,,1.0,house,,3 1,abbotsford,3067,2025,...,httpswwwdomaincomau130-charles-street-abbotsfo...,POINT (-37.8041915 144.9956615),875.0,"POLYGON ((144.983792 -37.809152, 144.984422 -3...","POLYGON ((144.961148 -37.805869, 144.96112 -37...","POLYGON ((144.924756 -37.829757, 144.925555 -3...",,,,POINT (-37.8041915 144.9956615)
3,17739186,1,,1.0,apartment unit flat,,1 1,abbotsford,3067,2025,...,httpswwwdomaincomau230-631-victoria-street-abb...,POINT (-37.8111571 145.008908),625.0,"POLYGON ((144.978882 -37.805563, 144.978845 -3...","POLYGON ((144.957292 -37.814043, 144.957299 -3...","POLYGON ((144.937243 -37.786821, 144.937272 -3...",,,,POINT (-37.8111571 145.008908)
4,17757992,2,,1.0,apartment unit flat,,2 1 1,abbotsford,3067,2025,...,httpswwwdomaincomau204-11-flockhart-street-abb...,POINT (-37.8109899 145.0067059),580.0,"POLYGON ((144.983792 -37.809152, 144.984422 -3...","POLYGON ((144.961148 -37.805869, 144.96112 -37...","POLYGON ((144.924756 -37.829757, 144.925555 -3...",,,,POINT (-37.8109899 145.0067059)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,17740732,4,,2.0,house,,4 2 2,boisdale,3860,2025,...,httpswwwdomaincomau199-boisdale-valencia-creek...,POINT (-37.8626779 146.987445),500.0,"POLYGON ((147.214933 -36.775441, 147.215281 -3...","POLYGON ((147.19283 -36.750824, 147.192931 -36...","POLYGON ((147.167961 -36.743249, 147.169208 -3...",,,,POINT (-37.8626779 146.987445)
496,17720600,4,,2.0,house,,4 2 2,bonbeach,3196,2025,...,httpswwwdomaincomau22-breeze-street-bonbeach-v...,POINT (-38.0678958 145.1241555),700.0,"POLYGON ((146.96751 -37.848101, 146.971103 -37...","POLYGON ((146.939632 -37.844354, 146.941434 -3...","POLYGON ((146.912508 -37.840832, 146.916004 -3...",,,,POINT (-38.0678958 145.1241555)
497,17753072,2,,2.0,apartment unit flat,,2 2 1,bonbeach,3196,2025,...,httpswwwdomaincomau2-15-bondi-road-bonbeach-vi...,POINT (-38.0627649 145.1219608),570.0,"POLYGON ((145.109838 -38.04644, 145.110028 -38...","POLYGON ((145.092518 -38.017424, 145.094211 -3...","POLYGON ((145.073442 -38.000271, 145.074379 -3...",,,,POINT (-38.0627649 145.1219608)
498,16329538,3,,2.0,house,,3 2 3 752m²,bonbeach,3196,2025,...,httpswwwdomaincomau2-480-481-station-street-bo...,POINT (-38.0720074 145.12213),699.0,"POLYGON ((145.105053 -38.038629, 145.105422 -3...","POLYGON ((145.086441 -38.012184, 145.092518 -3...","POLYGON ((145.064537 -37.993509, 145.070102 -3...",,,,POINT (-38.0720074 145.12213)


In [73]:
gdf_tmp

,property_id,bedrooms,bathrooms,car_spaces,property_type,land_area,property_features,suburb,postcode,year,...,url,coordinates,weekly_rent,driving_5min,driving_10min,driving_15min,walking_5min,walking_10min,walking_15min,geometry
0,17757280,2,,1.0,house,,2 1,abbotsford,3067,2025,...,httpswwwdomaincomau42-raphael-street-abbotsfor...,POINT (-37.8058235 144.9940691),850.0,,,,,,,POINT (-37.8058235 144.9940691)
1,17706353,1,,1.0,apartment unit flat,,1 1,abbotsford,3067,2025,...,httpswwwdomaincomau629-631-victoria-street-abb...,POINT (-37.8111571 145.008908),520.0,,,,,,,POINT (-37.8111571 145.008908)
2,17719363,3,,1.0,house,,3 1,abbotsford,3067,2025,...,httpswwwdomaincomau130-charles-street-abbotsfo...,POINT (-37.8041915 144.9956615),875.0,,,,,,,POINT (-37.8041915 144.9956615)
3,17739186,1,,1.0,apartment unit flat,,1 1,abbotsford,3067,2025,...,httpswwwdomaincomau230-631-victoria-street-abb...,POINT (-37.8111571 145.008908),625.0,,,,,,,POINT (-37.8111571 145.008908)
4,17757992,2,,1.0,apartment unit flat,,2 1 1,abbotsford,3067,2025,...,httpswwwdomaincomau204-11-flockhart-street-abb...,POINT (-37.8109899 145.0067059),580.0,,,,,,,POINT (-37.8109899 145.0067059)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,17740732,4,,2.0,house,,4 2 2,boisdale,3860,2025,...,httpswwwdomaincomau199-boisdale-valencia-creek...,POINT (-37.8626779 146.987445),500.0,,,,,,,POINT (-37.8626779 146.987445)
496,17720600,4,,2.0,house,,4 2 2,bonbeach,3196,2025,...,httpswwwdomaincomau22-breeze-street-bonbeach-v...,POINT (-38.0678958 145.1241555),700.0,,,,,,,POINT (-38.0678958 145.1241555)
497,17753072,2,,2.0,apartment unit flat,,2 2 1,bonbeach,3196,2025,...,httpswwwdomaincomau2-15-bondi-road-bonbeach-vi...,POINT (-38.0627649 145.1219608),570.0,,,,,,,POINT (-38.0627649 145.1219608)
498,16329538,3,,2.0,house,,3 2 3 752m²,bonbeach,3196,2025,...,httpswwwdomaincomau2-480-481-station-street-bo...,POINT (-38.0720074 145.12213),699.0,,,,,,,POINT (-38.0720074 145.12213)


In [70]:
result

0      [POLYGON ((144.976522 -37.810288, 144.978183 -...
1      [POLYGON ((144.983792 -37.809152, 144.984422 -...
2      [POLYGON ((144.978882 -37.805563, 144.978845 -...
3      [POLYGON ((144.983792 -37.809152, 144.984422 -...
4      [POLYGON ((144.979992 -37.808956, 144.980937 -...
                             ...                        
494    [POLYGON ((147.214933 -36.775441, 147.215281 -...
495    [POLYGON ((146.96751 -37.848101, 146.971103 -3...
496    [POLYGON ((145.109838 -38.04644, 145.110028 -3...
497    [POLYGON ((145.105053 -38.038629, 145.105422 -...
498    [POLYGON ((145.108349 -38.044669, 145.109945 -...
Name: geometry, Length: 499, dtype: object

In [67]:
gdf_tmp.head()

,property_id,bedrooms,bathrooms,car_spaces,property_type,land_area,property_features,suburb,postcode,year,...,url,coordinates,weekly_rent,driving_5min,driving_10min,driving_15min,walking_5min,walking_10min,walking_15min,geometry
0,17757280,2,,1.0,house,,2 1,abbotsford,3067,2025,...,httpswwwdomaincomau42-raphael-street-abbotsfor...,POINT (-37.8058235 144.9940691),850.0,,,,,,,POINT (-37.8058235 144.9940691)
1,17706353,1,,1.0,apartment unit flat,,1 1,abbotsford,3067,2025,...,httpswwwdomaincomau629-631-victoria-street-abb...,POINT (-37.8111571 145.008908),520.0,,,,,,,POINT (-37.8111571 145.008908)
2,17719363,3,,1.0,house,,3 1,abbotsford,3067,2025,...,httpswwwdomaincomau130-charles-street-abbotsfo...,POINT (-37.8041915 144.9956615),875.0,,,,,,,POINT (-37.8041915 144.9956615)
3,17739186,1,,1.0,apartment unit flat,,1 1,abbotsford,3067,2025,...,httpswwwdomaincomau230-631-victoria-street-abb...,POINT (-37.8111571 145.008908),625.0,,,,,,,POINT (-37.8111571 145.008908)
4,17757992,2,,1.0,apartment unit flat,,2 1 1,abbotsford,3067,2025,...,httpswwwdomaincomau204-11-flockhart-street-abb...,POINT (-37.8109899 145.0067059),580.0,,,,,,,POINT (-37.8109899 145.0067059)


In [71]:
# Split the result series (list of 3 POLYGON objects) into a DataFrame with 3 columns
# Method 1: Using pd.DataFrame() with tolist() - Most efficient
polygon_df = pd.DataFrame(result.tolist(), columns=['driving_5min', 'driving_10min', 'driving_15min'])

print("Shape of polygon_df:", polygon_df.shape)
print("Column names:", polygon_df.columns.tolist())
print("First few rows:")
print(polygon_df.head())
print("\nData types:")
print(polygon_df.dtypes)


Shape of polygon_df: (499, 3)
Column names: ['driving_5min', 'driving_10min', 'driving_15min']
First few rows:
                                        driving_5min  \
0  POLYGON ((144.976522 -37.810288, 144.978183 -3...   
1  POLYGON ((144.983792 -37.809152, 144.984422 -3...   
2  POLYGON ((144.978882 -37.805563, 144.978845 -3...   
3  POLYGON ((144.983792 -37.809152, 144.984422 -3...   
4  POLYGON ((144.979992 -37.808956, 144.980937 -3...   

                                       driving_10min  \
0  POLYGON ((144.954075 -37.805642, 144.954154 -3...   
1  POLYGON ((144.961148 -37.805869, 144.96112 -37...   
2  POLYGON ((144.957292 -37.814043, 144.957299 -3...   
3  POLYGON ((144.961148 -37.805869, 144.96112 -37...   
4  POLYGON ((144.957278 -37.814013, 144.957299 -3...   

                                       driving_15min  
0  POLYGON ((144.936515 -37.785747, 144.936694 -3...  
1  POLYGON ((144.924756 -37.829757, 144.925555 -3...  
2  POLYGON ((144.937243 -37.786821, 144.937272 -3.